In [4]:
from functools import reduce
import json
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 100)
pd.options.display.float_format = "{:,.2f}".format

## Import and format the data

Concatenate the data into a single file.

In [5]:
!sh process_eeoc.sh

process_eeoc.sh: 3: process_eeoc.sh: cannot create data/raw/charges_11_17.txt: Directory nonexistent
process_eeoc.sh: 4: process_eeoc.sh: cannot create data/raw/charges_11_17.txt: Directory nonexistent
process_eeoc.sh: 5: process_eeoc.sh: cannot create data/raw/charges_11_17.txt: Directory nonexistent
process_eeoc.sh: 6: process_eeoc.sh: cannot create data/raw/charges_11_17.txt: Directory nonexistent
process_eeoc.sh: 7: process_eeoc.sh: cannot create data/raw/charges_11_17.txt: Directory nonexistent
process_eeoc.sh: 8: process_eeoc.sh: cannot create data/raw/charges_11_17.txt: Directory nonexistent
process_eeoc.sh: 9: process_eeoc.sh: cannot create data/raw/charges_11_17.txt: Directory nonexistent
process_eeoc.sh: 10: process_eeoc.sh: cannot create data/raw/charges_11_17.txt: Directory nonexistent


Import charge data for fiscal years 2011-2017.

In [12]:
charges = pd.read_csv("data/raw/charges_11_17.txt", sep="\t", skiprows=1,
                      dtype={1: str},
                      names=["fiscal_year", "charge_num", "state", "num_employees_code",
                             "num_employees", "naics_code", "naics_desc", "type_code",
                             "type", "birth_date", "sex", "date_received", "date_closed",
                             "closure_code", "closure_action", "monetary_benefits", "statute_code",
                             "statute", "basis_code", "basis", "issue_code", "issue",
                             "court_filing_date", "civil_action_num", "court", "resolution_date",
                             "case_type", "litigation_monetary_benefits"])
charges.info()

FileNotFoundError: [Errno 2] File b'data/raw/charges_11_17.txt' does not exist: b'data/raw/charges_11_17.txt'

Convert the date columns.

In [13]:
charges["birth_date"] = pd.to_datetime(charges["birth_date"], errors="coerce", format="%m/%d/%Y")
charges["date_received"] = pd.to_datetime(charges["date_received"], errors="coerce", format="%m/%d/%Y")
charges["date_closed"] = pd.to_datetime(charges["date_closed"], errors="coerce", format="%m/%d/%Y")
charges["court_filing_date"] = pd.to_datetime(charges["court_filing_date"], errors="coerce", format="%m/%d/%Y")
charges["resolution_date"] = pd.to_datetime(charges["resolution_date"], errors="coerce", format="%m/%d/%Y")
charges.info()

NameError: name 'charges' is not defined

Import charge data for fiscal year 2010.

In [14]:
charges_10 = pd.read_csv("data/complaints_10.txt", sep="\t", skiprows=1,
                         dtype={0: str},
                         names=["charge_num", "state", "num_employees_code",
                                "num_employees", "naics_code", "naics_desc", "type_code",
                                "type", "birth_date", "sex", "date_received", "date_fepa_sent_to_eeoc",
                                "date_closed", "closure_code", "closure_action", "monetary_benefits",
                                "statute_code", "statute", "basis_code", "basis", "issue_code", "issue",
                                "court_filing_date", "civil_action_num", "court", "resolution_date",
                                "litigation_monetary_benefits", "case_type"])
charges_10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343863 entries, 0 to 343862
Data columns (total 28 columns):
charge_num                      343863 non-null object
state                           343801 non-null object
num_employees_code              328035 non-null object
num_employees                   328035 non-null object
naics_code                      187222 non-null float64
naics_desc                      185282 non-null object
type_code                       343829 non-null object
type                            343829 non-null object
birth_date                      310000 non-null object
sex                             343453 non-null object
date_received                   343863 non-null object
date_fepa_sent_to_eeoc          20188 non-null object
date_closed                     230050 non-null object
closure_code                    230050 non-null object
closure_action                  230050 non-null object
monetary_benefits               37964 non-null object
statute_co

Convert the date columns.

In [15]:
charges_10["birth_date"] = pd.to_datetime(charges_10["birth_date"], errors="coerce", format="%m/%d/%y")
charges_10["date_received"] = pd.to_datetime(charges_10["date_received"], errors="coerce", format="%m/%d/%y")
charges_10["date_closed"] = pd.to_datetime(charges_10["date_closed"], errors="coerce", format="%m/%d/%y")
charges_10["court_filing_date"] = pd.to_datetime(charges_10["court_filing_date"], errors="coerce", format="%m/%d/%y")
charges_10["resolution_date"] = pd.to_datetime(charges_10["resolution_date"], errors="coerce", format="%m/%d/%y")
charges_10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343863 entries, 0 to 343862
Data columns (total 28 columns):
charge_num                      343863 non-null object
state                           343801 non-null object
num_employees_code              328035 non-null object
num_employees                   328035 non-null object
naics_code                      187222 non-null float64
naics_desc                      185282 non-null object
type_code                       343829 non-null object
type                            343829 non-null object
birth_date                      310000 non-null datetime64[ns]
sex                             343453 non-null object
date_received                   343863 non-null datetime64[ns]
date_fepa_sent_to_eeoc          20188 non-null object
date_closed                     230050 non-null datetime64[ns]
closure_code                    230050 non-null object
closure_action                  230050 non-null object
monetary_benefits               37964 no

The columns in the 2010 charge data differ from those of the 2011 through 2017 data. We need to delete, add, rename and reorder the columns before concatenating the data.

In [16]:
charges_10.drop("date_fepa_sent_to_eeoc", axis=1, inplace=True)
charges_10["fiscal_year"] = "FY2010"
charges_10 = charges_10[["fiscal_year", "charge_num", "state", "num_employees_code", "num_employees", "naics_code", "naics_desc", "type_code",
                         "type", "birth_date", "sex", "date_received", "date_closed", "closure_code", "closure_action", "monetary_benefits",
                         "statute_code", "statute", "basis_code", "basis", "issue_code", "issue", "court_filing_date", "civil_action_num",
                         "court", "resolution_date", "case_type", "litigation_monetary_benefits"]]
charges_10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343863 entries, 0 to 343862
Data columns (total 28 columns):
fiscal_year                     343863 non-null object
charge_num                      343863 non-null object
state                           343801 non-null object
num_employees_code              328035 non-null object
num_employees                   328035 non-null object
naics_code                      187222 non-null float64
naics_desc                      185282 non-null object
type_code                       343829 non-null object
type                            343829 non-null object
birth_date                      310000 non-null datetime64[ns]
sex                             343453 non-null object
date_received                   343863 non-null datetime64[ns]
date_closed                     230050 non-null datetime64[ns]
closure_code                    230050 non-null object
closure_action                  230050 non-null object
monetary_benefits               37964 n

Concatenate the two sets of charge data.

In [19]:
# Added by Steve to make this kind of work
charges = pd.DataFrame()

In [20]:
charges = pd.concat([charges, charges_10], ignore_index=True)
charges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343863 entries, 0 to 343862
Data columns (total 28 columns):
fiscal_year                     343863 non-null object
charge_num                      343863 non-null object
state                           343801 non-null object
num_employees_code              328035 non-null object
num_employees                   328035 non-null object
naics_code                      187222 non-null float64
naics_desc                      185282 non-null object
type_code                       343829 non-null object
type                            343829 non-null object
birth_date                      310000 non-null datetime64[ns]
sex                             343453 non-null object
date_received                   343863 non-null datetime64[ns]
date_closed                     230050 non-null datetime64[ns]
closure_code                    230050 non-null object
closure_action                  230050 non-null object
monetary_benefits               37964 n

Convert the columns to their proper data types.

In [21]:
charges = charges.astype(dtype={"fiscal_year": "category", "charge_num": str, "state": "category",
                            "num_employees_code": "category", "num_employees": "category",
                            "naics_code": "category", "naics_desc": "category",
                            "type_code": "category", "type": "category", "sex": "category",
                            "closure_code": "category", "closure_action": "category",
                            "statute_code": "category", "statute": "category",
                            "basis_code": "category", "basis": "category", "issue_code": "category",
                            "issue": "category", "court": "category", "case_type": "category"})
charges["monetary_benefits"] = pd.to_numeric(charges["monetary_benefits"], errors="coerce", downcast="float")
charges["litigation_monetary_benefits"] = pd.to_numeric(charges["litigation_monetary_benefits"], errors="coerce", downcast="float")
charges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343863 entries, 0 to 343862
Data columns (total 28 columns):
fiscal_year                     343863 non-null category
charge_num                      343863 non-null object
state                           343801 non-null category
num_employees_code              328035 non-null category
num_employees                   328035 non-null category
naics_code                      187222 non-null category
naics_desc                      185282 non-null category
type_code                       343829 non-null category
type                            343829 non-null category
birth_date                      310000 non-null datetime64[ns]
sex                             343453 non-null category
date_received                   343863 non-null datetime64[ns]
date_closed                     230050 non-null datetime64[ns]
closure_code                    230050 non-null category
closure_action                  230050 non-null category
monetary_benefits 

Create a grouped basis column.

*Added by Steve*: Ok, what I think this is doing is if the basis column contains age, then we'll stick with Age. All the trues get age, now for the falses. If it contains color, then we'll go with color. If it doesn't contain age or color, 

In [22]:
charges["grouped_basis"] = np.where(charges["basis"].str.contains("age", case=False), "Age",
                           np.where(charges["basis"].str.contains("color", case=False), "Color",
                           np.where(charges["basis"].isin(["Alcoholism", "Allergies", "Alzheimers",
                                                         "Asthma", "Autism", "Blood (Other)",
                                                         "Brain/Head Impairment", "Brain/Head Injury (Traumatic)",
                                                         "Cancer", "Cerebral Palsy", "Chemical Sensitivity",
                                                         "Cumulative Trauma Disorder", "Cystic Fibrosis",
                                                         "Depression", "Diabetes", "Disfigurement",
                                                         "Drug Addiction", "Dwarfism", "Epilepsy",
                                                         "Gastrointestinal", "HIV", "Handicap (Not ADA)",
                                                         "Hearing Impairment", "Heart/Cardiovascular",
                                                         "Intellectual Disability", "Kidney Impairment",
                                                         "Learning Disability", "Manic Depression (Bi-polar)",
                                                         "Missing Digits/Limbs", "Multiple Sclerosis",
                                                         "Nonparalytic Orthopedic Impairment",
                                                         "Orthopedic/Structural Back Impairment",
                                                         "Other Anxiety Disorder", "Other Disability",
                                                         "Other Neurological", "Other Psychiatric Disorders",
                                                         "Other Pulmo/Respiratory", "Paralysis",
                                                         "Post-Traumatic Stress Disorder", "Record Of Disability",
                                                         "Regarded As Disabled", "Schizophrenia",
                                                         "Speech Impairment", "Tuberculosis", "Vision Impairment"]), "Disability/Medical",
                            np.where(charges["basis"].str.contains("equal pay", case=False), "Equal Pay",
                            np.where(charges["basis"].str.contains("genetic", case=False), "Genetics",
                            np.where(charges["basis"].str.contains("ancestry", case=False) | charges["basis"].str.contains("national origin", case=False), "Ancestry/National Origin",
                            np.where(charges["basis"].isin(["Conviction Record", "Marital Status", "Other",
                                                            "Relationship/Assn.", "Unassigned"]), "Other",
                            np.where(charges["basis"].str.contains("race", case=False), "Race",
                            np.where(charges["basis"].str.contains("retaliation", case=False), "Retaliation",
                            np.where(charges["basis"].str.contains("religion", case=False), "Religion",
                            np.where(charges["basis"].str.contains("sex", case=False), "Sex",
                                    "")))))))))))
charges.groupby("grouped_basis")["charge_num"].count()

grouped_basis
Age                         38553
Ancestry/National Origin    23393
Color                        6064
Disability/Medical          59399
Equal Pay                    1523
Genetics                      355
Other                        2822
Race                        66297
Religion                     7383
Retaliation                 81570
Sex                         56504
Name: charge_num, dtype: int64

In [29]:
charges.head()

,fiscal_year,charge_num,state,num_employees_code,num_employees,naics_code,naics_desc,type_code,type,birth_date,sex,date_received,date_closed,closure_code,closure_action,monetary_benefits,statute_code,statute,basis_code,basis,issue_code,issue,court_filing_date,civil_action_num,court,resolution_date,case_type,litigation_monetary_benefits,grouped_basis
0,FY2010,"3,288,411.72",MD,B,101 - 200 Employees,"611,110.00",Elementary and Secondary Schools,E,Private Employer,2066-11-13,N,2010-03-03,2010-03-03,M3,No Cause Finding Issued,nan,T,Title VII,OR,Retaliation,T2,Terms/Conditions,NaT,NaN,NaN,NaT,NaN,nan,Retaliation
1,FY2010,"3,288,411.72",MD,B,101 - 200 Employees,"611,110.00",Elementary and Secondary Schools,E,Private Employer,2066-11-13,N,2010-03-03,2010-03-03,M3,No Cause Finding Issued,nan,A,ADEA,OR,Retaliation,T2,Terms/Conditions,NaT,NaN,NaN,NaT,NaN,nan,Retaliation
2,FY2010,"3,288,411.72",MD,B,101 - 200 Employees,"611,110.00",Elementary and Secondary Schools,E,Private Employer,2066-11-13,N,2010-03-03,2010-03-03,M3,No Cause Finding Issued,nan,A,ADEA,OA,Age,T2,Terms/Conditions,NaT,NaN,NaN,NaT,NaN,nan,Age
3,FY2010,"3,593,445.5595",AL,U,Unknown Number Of Employees,nan,NaN,E,Private Employer,1979-05-02,F,2010-02-03,2010-07-30,N2,NRTS Issued At CP Request,nan,T,Title VII,OR,Retaliation,D2,Discharge,NaT,NaN,NaN,NaT,NaN,nan,Retaliation
4,FY2010,"3,593,445.5595",AL,U,Unknown Number Of Employees,nan,NaN,E,Private Employer,1979-05-02,F,2010-02-03,2010-07-30,N2,NRTS Issued At CP Request,nan,T,Title VII,OR,Retaliation,B3,Benefits-Insurance,NaT,NaN,NaN,NaT,NaN,nan,Retaliation


## How many alleged violations are we dealing with?

In [ ]:
charges["charge_num"].count()

## And how many cases are we dealing with?

In [ ]:
charges["charge_num"].nunique()

How many of these cases occurred in each year?

In [ ]:
all_cases_by_year = charges.groupby("fiscal_year")["charge_num"].nunique().reset_index()
all_cases_by_year.rename(columns={"charge_num": "all_cases"}, inplace=True)
all_cases_by_year

How does this break down by basis?

In [ ]:
all_cases_by_basis = charges.groupby(["basis"])["charge_num"].nunique().reset_index()
all_cases_by_basis.rename(columns={"charge_num": "all_cases"}, inplace=True)
all_cases_by_basis.sort_values("all_cases", ascending=False).head()

How does this break down by grouped basis?

In [ ]:
all_cases_by_grouped_basis = charges.groupby(["grouped_basis"])["charge_num"].nunique().reset_index()
all_cases_by_grouped_basis.rename(columns={"charge_num": "all_cases"}, inplace=True)
all_cases_by_grouped_basis.sort_values("all_cases", ascending=False).head()

## How many closed cases are we dealing with?

In [ ]:
closed_charges = charges[charges["closure_action"].notnull()]
closed_cases = charges[charges["closure_action"].notnull()]
closed_cases["charge_num"].nunique()

And how many of these cases were essentially dismissed outright?

In [ ]:
closed_cases[closed_cases["closure_action"] == "No Cause Finding Issued"]["charge_num"].nunique()

In [ ]:
closed_cases[closed_cases["closure_action"] == "No Cause Finding Issued"]["charge_num"].nunique() / closed_cases["charge_num"].nunique()

How many of these occurred in each year?

In [ ]:
closed_cases_by_year = closed_cases.groupby("fiscal_year")["charge_num"].nunique().reset_index()
closed_cases_by_year.rename(columns={"charge_num": "closed_cases"}, inplace=True)
closed_cases_by_year

How does this break down by basis?

In [ ]:
closed_cases_by_basis = closed_cases.groupby(["basis"])["charge_num"].nunique().reset_index()
closed_cases_by_basis.rename(columns={"charge_num": "closed_cases"}, inplace=True)
closed_cases_by_basis.sort_values("closed_cases", ascending=False).head()

How does this break down by grouped basis?

In [ ]:
closed_cases_by_grouped_basis = closed_cases.groupby(["grouped_basis"])["charge_num"].nunique().reset_index()
closed_cases_by_grouped_basis.rename(columns={"charge_num": "closed_cases"}, inplace=True)
closed_cases_by_grouped_basis.sort_values("closed_cases", ascending=False).head()

## In how many cases did the EEOC find merit to the complaint?

In [ ]:
meritorious_outcomes = ["Case Settled By Legal Unit", "Conciliation Failure", "Successful Conciliation"]
meritorious_cases = closed_cases[closed_cases["closure_action"].isin(meritorious_outcomes)]
meritorious_cases["charge_num"].nunique()

In [ ]:
meritorious_cases["charge_num"].nunique() / closed_cases["charge_num"].nunique()

Not every case deemed meritorious by the agencies resulted in a worker receiving some form of relief. In how many such cases did a worker not receive relief?

In [ ]:
meritorious_cases[meritorious_cases["closure_action"] == "Conciliation Failure"]["charge_num"].nunique() / meritorious_cases["charge_num"].nunique()

How many meritorious cases occurred in each year?

In [ ]:
meritorious_cases_by_year = meritorious_cases.groupby("fiscal_year")["charge_num"].nunique().reset_index()
meritorious_cases_by_year.rename(columns={"charge_num": "meritorious_cases"}, inplace=True)
meritorious_cases_by_year

How does this break down by basis?

In [ ]:
meritorious_cases_by_basis = meritorious_cases.groupby(["basis"])["charge_num"].nunique().reset_index()
meritorious_cases_by_basis.rename(columns={"charge_num": "meritorious_cases"}, inplace=True)
meritorious_cases_by_basis.sort_values("meritorious_cases", ascending=False).head()

How does this break down by grouped basis?

In [ ]:
meritorious_cases_by_grouped_basis = meritorious_cases.groupby(["grouped_basis"])["charge_num"].nunique().reset_index()
meritorious_cases_by_grouped_basis.rename(columns={"charge_num": "meritorious_cases"}, inplace=True)
meritorious_cases_by_grouped_basis.sort_values("meritorious_cases", ascending=False).head()

## And in how many cases did the EEOC grant some form of relief (including non-monetary relief) to the complainant?

In [ ]:
relief_outcomes = ["Case Settled By Legal Unit", "Settlement With Benefits", "Successful Conciliation", "Withdrawal With Benefits"]
relief_cases = closed_cases[closed_cases["closure_action"].isin(relief_outcomes)]
relief_cases["charge_num"].nunique()

In [ ]:
relief_cases["charge_num"].nunique() / closed_cases["charge_num"].nunique()

How many of these cases occurred in each year?

In [ ]:
relief_cases_by_year = relief_cases.groupby("fiscal_year")["charge_num"].nunique().reset_index()
relief_cases_by_year.rename(columns={"charge_num": "relief_cases"}, inplace=True)
relief_cases_by_year

How does this break down by basis?

In [ ]:
relief_cases_by_basis = relief_cases.groupby("basis")["charge_num"].nunique().reset_index()
relief_cases_by_basis.rename(columns={"charge_num": "relief_cases"}, inplace=True)
relief_cases_by_basis.sort_values("relief_cases", ascending=False).head()

How does this break down by grouped basis?

In [ ]:
relief_cases_by_grouped_basis = relief_cases.groupby("grouped_basis")["charge_num"].nunique().reset_index()
relief_cases_by_grouped_basis.rename(columns={"charge_num": "relief_cases"}, inplace=True)
relief_cases_by_grouped_basis.sort_values("relief_cases", ascending=False).head()

## And in how many cases did a worker see any monetary benefits?

In [ ]:
monetary_benefits_cases = closed_cases[closed_charges["monetary_benefits"] > 0]
monetary_benefits_cases["charge_num"].nunique()

In [ ]:
monetary_benefits_cases["charge_num"].nunique() / closed_cases["charge_num"].nunique()

How much money did they get?

In [ ]:
monetary_benefits_cases.groupby(["charge_num"])["monetary_benefits"].mean().sum()

How many of these cases occurred in each year?

In [ ]:
monetary_benefits_cases_by_year = monetary_benefits_cases.groupby("fiscal_year")["charge_num"].nunique().reset_index()
monetary_benefits_cases_by_year.rename(columns={"charge_num": "monetary_benefits_cases"}, inplace=True)
monetary_benefits_cases_by_year

How does this break down by basis?

In [ ]:
monetary_benefits_cases_by_basis = monetary_benefits_cases.groupby("basis")["charge_num"].nunique().reset_index()
monetary_benefits_cases_by_basis.rename(columns={"charge_num": "monetary_benefits_cases"}, inplace=True)
monetary_benefits_cases_by_basis.sort_values("monetary_benefits_cases", ascending=False).head()

How does this break down by grouped basis?

In [ ]:
monetary_benefits_cases_by_grouped_basis = monetary_benefits_cases.groupby("grouped_basis")["charge_num"].nunique().reset_index()
monetary_benefits_cases_by_grouped_basis.rename(columns={"charge_num": "monetary_benefits_cases"}, inplace=True)
monetary_benefits_cases_by_grouped_basis.sort_values("monetary_benefits_cases", ascending=False).head()

## How many cases alleged some form of racial discrimination?

In [ ]:
race_discrimination = charges[charges["grouped_basis"] == "Race"]
race_discrimination["charge_num"].nunique()

In [ ]:
race_discrimination["charge_num"].nunique() / charges["charge_num"].nunique()

## How many cases alleged racial discrimination against African-Americans?

In [ ]:
aa_discrimination = charges[charges["basis"] == "Race-Black/African American"]
aa_discrimination["charge_num"].nunique()

In [ ]:
aa_discrimination["charge_num"].nunique() / charges["charge_num"].nunique()

## Combine the data by year.

In [ ]:
cases_by_year_dfs = [all_cases_by_year, closed_cases_by_year, meritorious_cases_by_year, relief_cases_by_year, monetary_benefits_cases_by_year]
cases_by_year = reduce(lambda left, right: pd.merge(left, right, on="fiscal_year"), cases_by_year_dfs)
cases_by_year

Calculate the proportion of all cases that fall into each category each fiscal year.

In [ ]:
cases_by_year["pct_all_cases_closed"] = cases_by_year["closed_cases"] / cases_by_year["all_cases"]
cases_by_year["pct_closed_cases_meritorious"] = cases_by_year["meritorious_cases"] / cases_by_year["closed_cases"]
cases_by_year["pct_closed_cases_relief"] = cases_by_year["relief_cases"] / cases_by_year["closed_cases"]
cases_by_year["pct_closed_cases_monetary_benefits"] = cases_by_year["monetary_benefits_cases"] / cases_by_year["closed_cases"]
cases_by_year = cases_by_year[["fiscal_year", "all_cases", "closed_cases", "pct_all_cases_closed", "meritorious_cases", "pct_closed_cases_meritorious", "relief_cases", "pct_closed_cases_relief", "monetary_benefits_cases", "pct_closed_cases_monetary_benefits"]]
cases_by_year

## Combine the data by basis.

In [ ]:
cases_by_basis_dfs = [all_cases_by_basis, closed_cases_by_basis, meritorious_cases_by_basis, relief_cases_by_basis, monetary_benefits_cases_by_basis]
cases_by_basis = reduce(lambda left, right: pd.merge(left, right, on="basis"), cases_by_basis_dfs)
cases_by_basis.sort_values("all_cases", ascending=False).head()

Calculate the proportion of all cases that fall into each category by basis.

In [ ]:
cases_by_basis["pct_all_cases_closed"] = cases_by_basis["closed_cases"] / cases_by_basis["all_cases"]
cases_by_basis["pct_closed_cases_meritorious"] = cases_by_basis["meritorious_cases"] / cases_by_basis["closed_cases"]
cases_by_basis["pct_closed_cases_relief"] = cases_by_basis["relief_cases"] / cases_by_basis["closed_cases"]
cases_by_basis["pct_closed_cases_monetary_benefits"] = cases_by_basis["monetary_benefits_cases"] / cases_by_basis["closed_cases"]
cases_by_basis = cases_by_basis[["basis", "all_cases", "closed_cases", "pct_all_cases_closed", "meritorious_cases", "pct_closed_cases_meritorious", "relief_cases", "pct_closed_cases_relief", "monetary_benefits_cases", "pct_closed_cases_monetary_benefits"]]
cases_by_basis.sort_values("all_cases", ascending=False).head()

## Combine the data by grouped basis.

In [ ]:
cases_by_grouped_basis_dfs = [all_cases_by_grouped_basis, closed_cases_by_grouped_basis, meritorious_cases_by_grouped_basis, relief_cases_by_grouped_basis, monetary_benefits_cases_by_grouped_basis]
cases_by_grouped_basis = reduce(lambda left, right: pd.merge(left, right, on="grouped_basis"), cases_by_grouped_basis_dfs)
cases_by_grouped_basis.sort_values("all_cases", ascending=False).head()

Calculate the proportion of all cases that fall into each category by grouped basis.

In [ ]:
cases_by_grouped_basis["pct_all_cases_closed"] = cases_by_grouped_basis["closed_cases"] / cases_by_grouped_basis["all_cases"]
cases_by_grouped_basis["pct_closed_cases_meritorious"] = cases_by_grouped_basis["meritorious_cases"] / cases_by_grouped_basis["closed_cases"]
cases_by_grouped_basis["pct_closed_cases_relief"] = cases_by_grouped_basis["relief_cases"] / cases_by_grouped_basis["closed_cases"]
cases_by_grouped_basis["pct_closed_cases_monetary_benefits"] = cases_by_grouped_basis["monetary_benefits_cases"] / cases_by_grouped_basis["closed_cases"]
cases_by_grouped_basis = cases_by_grouped_basis[["grouped_basis", "all_cases", "closed_cases", "pct_all_cases_closed", "meritorious_cases", "pct_closed_cases_meritorious", "relief_cases", "pct_closed_cases_relief", "monetary_benefits_cases", "pct_closed_cases_monetary_benefits"]]
cases_by_grouped_basis.sort_values("all_cases", ascending=False).head()

In [ ]:
charges.groupby("charge_num")["date_received"].nunique().reset_index().sort_values("date_received", ascending=False).head(1)

In [ ]:
charges[charges["charge_num"] == "5688973"]

## What sorts of discriminatory bases were most likely to result in an outcome where the EEOC found merit in the complaint, the complainant got some form of relief and in which the complainant saw any monetary benefits and how does that compare with the overall number of those violations (minimum 100 closed cases)?

In [ ]:
cases_by_basis[cases_by_basis["closed_cases"] >= 100].sort_values("pct_closed_cases_meritorious", ascending=False).head()

In [ ]:
cases_by_basis[cases_by_basis["closed_cases"] >= 100].sort_values("pct_closed_cases_relief", ascending=False).head()

In [ ]:
cases_by_basis[cases_by_basis["closed_cases"] >= 100].sort_values("pct_closed_cases_monetary_benefits", ascending=False).head()

## What sorts of discriminatory grouped bases were most likely to result in an outcome where the EEOC found merit in the complaint, the complainant got some form of relief and in which the complainant saw any monetary benefits and how does that compare with the overall number of those violations (minimum 100 closed cases)?

In [ ]:
cases_by_grouped_basis[cases_by_grouped_basis["closed_cases"] >= 100].sort_values("pct_closed_cases_meritorious", ascending=False).head()

In [ ]:
cases_by_grouped_basis[cases_by_grouped_basis["closed_cases"] >= 100].sort_values("pct_closed_cases_relief", ascending=False).head()

In [ ]:
cases_by_grouped_basis[cases_by_grouped_basis["closed_cases"] >= 100].sort_values("pct_closed_cases_monetary_benefits", ascending=False).head()

## How many cases alleged some form of racial discrimination?

In [ ]:
race_discrimination = charges[charges["grouped_basis"] == "Race"]
race_discrimination["charge_num"].nunique()

In [ ]:
race_discrimination["charge_num"].nunique() / charges["charge_num"].nunique()

## What proportion of closed cases alleging some form of racial discrimination resulted in the complainant getting some form of relief?

In [ ]:
race_discrimination_relief_cases = closed_cases[(closed_cases["grouped_basis"] == "Race") & (closed_cases["closure_action"].isin(relief_outcomes))]
race_discrimination_relief_cases["charge_num"].nunique()

In [ ]:
race_discrimination_relief_cases["charge_num"].nunique() / closed_cases[closed_cases["grouped_basis"] == "Race"]["charge_num"].nunique()

## What proportion of closed cases not alleging some form of racial discrimination resulted in the complainant getting some form of relief?

Create a list of charge numbers attached to cases in which some form of racial discrimination was alleged.

In [ ]:
race_discrimination_charge_nums = race_discrimination["charge_num"].drop_duplicates().tolist()

Calculate the number of relief cases where racial discrimination was not alleged.

In [ ]:
non_race_discrimination_relief_cases = closed_cases[(~closed_cases["charge_num"].isin(race_discrimination_charge_nums)) & (closed_cases["closure_action"].isin(relief_outcomes))]
non_race_discrimination_relief_cases["charge_num"].nunique()

Calculate the number of all closed cases where racial discrimination was not alleged.

In [ ]:
non_race_discrimination_closed_cases = closed_cases[~closed_cases["charge_num"].isin(race_discrimination_charge_nums)]
non_race_discrimination_closed_cases["charge_num"].nunique()

In [ ]:
non_race_discrimination_relief_cases["charge_num"].nunique() / non_race_discrimination_closed_cases["charge_num"].nunique()

## How many cases alleged racial discrimination against African-Americans?

In [ ]:
aa_discrimination = charges[charges["basis"] == "Race-Black/African American"]
aa_discrimination["charge_num"].nunique()

In [ ]:
aa_discrimination["charge_num"].nunique() / charges["charge_num"].nunique()

## What does this data look like when grouped by basis and closure action?

In [ ]:
cases_by_basis_and_closure_action = closed_cases.groupby(["basis", "closure_action"])["charge_num"].nunique().reset_index()
cases_by_basis_and_closure_action.rename(columns={"charge_num": "cases"}, inplace=True)
cases_by_basis_and_closure_action.head()

In [ ]:
cases_by_basis_and_closure_action = pd.pivot_table(cases_by_basis_and_closure_action, index=["basis"], columns=["closure_action"])
cases_by_basis_and_closure_action.fillna(0, inplace=True)
cases_by_basis_and_closure_action.head()

In [ ]:
pct_case_by_basis_and_closure_action = cases_by_basis_and_closure_action.apply(lambda x: x/x.sum(), axis=1)
pct_case_by_basis_and_closure_action.rename(lambda x: x + "_pct", axis=1, inplace=True)

In [ ]:
pct_case_by_basis_and_closure_action.columns = ["_".join(column).replace(", ","_").lower().strip("_") for column in cases_by_basis_and_closure_action.columns.values]
pct_case_by_basis_and_closure_action.reset_index(inplace=True)

In [ ]:
cases_by_basis_and_closure_action.columns = ["_".join(column).replace(", ","_").lower().strip("_") for column in cases_by_basis_and_closure_action.columns.values]
cases_by_basis_and_closure_action.reset_index(inplace=True)

In [ ]:
cases_by_basis_and_closure_action = cases_by_basis_and_closure_action.merge(pct_case_by_basis_and_closure_action, on="basis", suffixes=["", "_pct"])
cases_by_basis_and_closure_action.head()

Export the tables.

In [ ]:
cases_by_year.to_csv("data/cases_by_year.csv", index=False)
cases_by_basis.to_csv("data/cases_by_basis.csv", index=False)
cases_by_grouped_basis.to_csv("data/cases_by_grouped_basis.csv", index=False)
cases_by_basis_and_closure_action.to_csv("data/cases_by_basis_and_closure_action.csv", index=False)